In [2]:
import numpy as np
import pandas as pd
import os

In [198]:
centmed = pd.read_csv(os.path.join("resources","base","centmed.csv"))
therapists = pd.read_csv(os.path.join("resources","base","therapists.csv"))
patients = pd.read_csv(os.path.join("resources","base","patients.csv"))

activities = pd.read_csv(os.path.join("resources","base","activities.csv"))
activities['DAY'] =  pd.to_datetime(activities['DAY'], format='%d/%m/%Y')
activities.set_index(['PATIENT', 'DAY','HOUR'], inplace=True)

emotions = pd.read_csv(os.path.join("resources","base","emotions.csv"))
emotions['DAY'] =  pd.to_datetime(emotions['DAY'], format='%d/%m/%Y')
emotions.set_index(['PATIENT', 'DAY','HOUR'], inplace=True)

episods = pd.read_csv(os.path.join("resources","base","episods.csv"))
episods.replace(np.nan,'NONE', regex=True, inplace=True)
episods['DAY'] =  pd.to_datetime(episods['DAY'], format='%d/%m/%Y')
episods.set_index(['PATIENT', 'DAY','HOUR'], inplace=True)

meals = pd.read_csv(os.path.join("resources","base","meals.csv"))
meals['DAY'] =  pd.to_datetime(meals['DAY'], format='%d/%m/%Y')
meals.set_index(['PATIENT', 'DAY','HOUR'], inplace=True)


print patients.shape, activities.shape, emotions.shape, episods.shape, meals.shape, 
print centmed.shape, therapists.shape

#activities['DAY'] =  pd.to_datetime(activities['DAY'])

(30, 5) (8280, 1) (8280, 1) (8280, 1) (8280, 1) (10, 4) (20, 2)


In [199]:
np.unique(dfPatients.EPISODE)

array([0, 1, 2, 3])

In [200]:
dfPatients = pd.concat([activities,emotions, episods,meals],join_axes=[activities.index], axis=1)
dfPatients.reset_index(level=[0,1,2], inplace=True)
mapping_0 = {'00:00 - 10:00':1, '10:00 - 16:00':2, '16:00 - 24:00':3}
mapping_1 = {'NONE':0, 'BINGE':1,'RESTRICTION':2, 'VOMITED':3}
dfPatients = dfPatients.replace({'HOUR':mapping_0})
dfPatients = dfPatients.replace({'EPISODE':mapping_1})

In [201]:
dfPatients.head()

,PATIENT,DAY,HOUR,ACTIVITY,EMOTION,EPISODE,MEAL
0,P1,2015-11-01,1,STUDYING/WORKING,SAD,2,NOTHING
1,P1,2015-11-01,2,SPORTS,RELAXED,0,3 THIRD
2,P1,2015-11-01,3,FAMILY,RELAXED,0,2 THIRD
3,P1,2015-11-02,1,FRIENDS,HAPPY,0,2 THIRD
4,P1,2015-11-02,2,FRIENDS,RELAXED,0,3 THIRD


In [202]:
dfPatientsVect = pd.get_dummies(dfPatients, columns=['ACTIVITY','EMOTION','MEAL'])
dfPatientsVect = dfPatientsVect.sort(columns=['PATIENT','DAY','HOUR'])
dfPatientsVect['WEEK']= dfPatientsVect.DAY.dt.week
print dfPatientsVect.columns
dfPatientsVect.head()

Index([u'PATIENT', u'DAY', u'HOUR', u'EPISODE', u'ACTIVITY_ALONE',
       u'ACTIVITY_FAMILY', u'ACTIVITY_FRIENDS', u'ACTIVITY_SPORTS',
       u'ACTIVITY_STUDYING/WORKING', u'EMOTION_DISTRESSED',
       u'EMOTION_FUSTRATED', u'EMOTION_GUILTY', u'EMOTION_HAPPY',
       u'EMOTION_RELAXED', u'EMOTION_SAD', u'MEAL_+  3 THIRD', u'MEAL_1 THIRD',
       u'MEAL_2 THIRD', u'MEAL_3 THIRD', u'MEAL_NOTHING', u'WEEK'],
      dtype='object')


/home/xrafael/Projects/hack/hackandhealth/venv/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,PATIENT,DAY,HOUR,EPISODE,ACTIVITY_ALONE,ACTIVITY_FAMILY,ACTIVITY_FRIENDS,ACTIVITY_SPORTS,ACTIVITY_STUDYING/WORKING,EMOTION_DISTRESSED,...,EMOTION_GUILTY,EMOTION_HAPPY,EMOTION_RELAXED,EMOTION_SAD,MEAL_+ 3 THIRD,MEAL_1 THIRD,MEAL_2 THIRD,MEAL_3 THIRD,MEAL_NOTHING,WEEK
0,P1,2015-11-01,1,2,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,44
1,P1,2015-11-01,2,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,44
2,P1,2015-11-01,3,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,44
3,P1,2015-11-02,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,45
4,P1,2015-11-02,2,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,45


In [214]:
dfPatients.to_csv(os.path.join("resources","base","joined.csv"))
dfPatientsVect.to_csv(os.path.join("resources","base","joinedVect.csv"))

#### Data Exploration

In [206]:
dfPatientsVectExp = pd.get_dummies(dfPatientsVect, columns=['EPISODE'])

res = dfPatientsVectExp[['PATIENT','WEEK','EPISODE_0','EPISODE_1','EPISODE_2','EPISODE_3']].groupby(by=['PATIENT','WEEK'])
resAll = res.agg(['sum'])
resAll.reset_index(level=[0,1], inplace=True)
resAll.sort(columns=['PATIENT','WEEK'],inplace=True)
resAll.to_csv(os.path.join("resources","base","groupPatientWeekEpis.csv"))

/home/xrafael/Projects/hack/hackandhealth/venv/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [211]:
resAllNoP = resAll[['WEEK','EPISODE_0','EPISODE_1','EPISODE_2','EPISODE_3']].groupby(by='WEEK')
resAllNoP = resAllNoP.agg(['sum'])
resAllNoP.to_csv(os.path.join("resources","base","groupWeekEpis.csv"))

In [121]:
dfPatientsVect[np.logical_and(dfPatientsVect['PATIENT'] == 'P2',dfPatientsVect['EPISODE'] == 3)]

,PATIENT,DAY,HOUR,EPISODE,ACTIVITY_ALONE,ACTIVITY_FAMILY,ACTIVITY_FRIENDS,ACTIVITY_SPORTS,ACTIVITY_STUDYING/WORKING,EMOTION_DISTRESSED,EMOTION_FUSTRATED,EMOTION_GUILTY,EMOTION_HAPPY,EMOTION_RELAXED,EMOTION_SAD,MEAL_+ 3 THIRD,MEAL_1 THIRD,MEAL_2 THIRD,MEAL_3 THIRD,MEAL_NOTHING
280,P2,2015-02-11,2,3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
288,P2,2015-05-11,1,3,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
302,P2,2015-09-11,3,3,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0
317,P2,2015-11-14,3,3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
330,P2,2015-11-19,1,3,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
339,P2,2015-11-22,1,3,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
351,P2,2015-11-26,1,3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1
361,P2,2015-11-29,2,3,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
373,P2,2015-03-12,2,3,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0
387,P2,2015-08-12,1,3,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


In [212]:
resAll.head()

,PATIENT,WEEK,EPISODE_0,EPISODE_1,EPISODE_2,EPISODE_3
,,,sum,sum,sum,sum
0,P1,1,17,2,2,0
1,P1,2,18,0,3,0
2,P1,3,16,0,5,0
3,P1,4,14,0,7,0
4,P1,44,2,0,1,0


In [213]:
resAllNoP.head()

,EPISODE_0,EPISODE_1,EPISODE_2,EPISODE_3
,sum,sum,sum,sum
,sum,sum,sum,sum
WEEK,,,,
1,523.0,33,40,34
2,515.0,26,45,44
3,528.0,21,40,41
4,507.0,20,61,42
44,48.0,22,10,10


#### Aggregations random

In [144]:
res = dfPatientsVect.groupby(by=['PATIENT','EPISODE'])
res.agg(['sum'])

HOUR ACTIVITY_ALONE ACTIVITY_FAMILY ACTIVITY_FRIENDS  \
                 sum            sum             sum              sum   
PATIENT EPISODE                                                        
P1      0        441             21              43               48   
        1         20              5               0                1   
        2         91             17               0                7   
P10     0        455              0              50               53   
        1         24             11               0                0   
        3         73             12               0                5   
P11     0        464              0              42               55   
        1         35             17               0                0   
        3         53              9               0                4   
P12     0        462              0              44               56   
        1         22             13               0                0   
        2         68             19               0                7   
P13     0        422              0              48               38   
        1         36             17               0                0   
        3         94             23               0               12   
P14     0        471              0              39               47   
        1         26             13               0                0   
        2         55             13               0                7   
P15     0        474              0              42               43   
        1         17             11               0                0   
        3         61             17               0                6   
P16     0        397              0              40               40   
        1         21             13               0                0   
        2        134             24               0               15   
P17     0        442              0              46               45   
        1         26             15               0                0   
        3         84             14               0                6   
P18     0        444              0              42               39   
        1         27             14               0                0   
        2         81             14               0                6   
...              ...            ...             ...              ...   
P28     1         34             15               0                0   
        3        101             21               0               10   
P29     0        497              0              42               52   
        1         21             11               0                0   
        3         34             10               0                2   
P3      0        457              1              40               49   
        1         30             10               0                2   
        2         65             10               0                7   
P30     0        499              0              56               46   
        1         13              8               0                0   
        2         40              8               0                3   
P4      0        449              0              38               51   
        1         34             20               0                0   
        2          7              1               0                1   
        3         62             11               0                7   
P5      0        504              0              54               49   
        1         17             10               0                0   
        3         31              4               0                5   
P6      0        471              0              42               47   
        1         22             10               0                0   
        2         59             15               0                6   
P7      0        459              0              47     

In [136]:
res = dfPatientsVect[dfPatientsVect['EPISODE'] == 3].groupby(by=['PATIENT','EPISODE'])
res.agg(['count','mean','std'])[['ACTIVITY_ALONE','ACTIVITY_FAMILY','ACTIVITY_FRIENDS','ACTIVITY_SPORTS','ACTIVITY_STUDYING/WORKING']]\
.head(10)

ACTIVITY_ALONE                     ACTIVITY_FAMILY            \
                         count      mean       std           count mean  std   
PATIENT EPISODE                                                                
P10     3                   35  0.342857  0.481594              35    0  0.0   
P11     3                   25  0.360000  0.489898              25    0  0.0   
P13     3                   52  0.442308  0.501506              52    0  0.0   
P15     3                   32  0.531250  0.507007              32    0  0.0   
P17     3                   40  0.350000  0.483046              40    0  0.0   
P19     3                    7  0.571429  0.534522               7    0  0.0   
P2      3                   17  0.588235  0.507300              17    0  0.0   
P20     3                   42  0.428571  0.500870              42    0  0.0   
P21     3                    1  0.000000       NaN               1    0  NaN   
P22     3                   31  0.451613  0.505879              31    0  0.0   

                ACTIVITY_FRIENDS                     ACTIVITY_SPORTS       \
                           count      mean       std           count mean   
PATIENT EPISODE                                                             
P10     3                     35  0.142857  0.355036              35    0   
P11     3                     25  0.160000  0.374166              25    0   
P13     3                     52  0.230769  0.425436              52    0   
P15     3                     32  0.187500  0.396558              32    0   
P17     3                     40  0.150000  0.361620              40    0   
P19     3                      7  0.000000  0.000000               7    0   
P2      3                     17  0.117647  0.332106              17    0   
P20     3                     42  0.190476  0.397437              42    0   
P21     3                      1  1.000000       NaN               1    0   
P22     3                     31  0.290323  0.461414              31    0   

                     ACTIVITY_STUDYING/WORKING                      
                 std                     count      mean       std  
PATIENT EPISODE                                                     
P10     3        0.0                        35  0.514286  0.507093  
P11     3        0.0                        25  0.480000  0.509902  
P13     3        0.0                        52  0.326923  0.473665  
P15     3        0.0                        32  0.281250  0.456803  
P17     3        0.0                        40  0.500000  0.506370  
P19     3        0.0                         7  0.428571  0.534522  
P2      3        0.0                        17  0.294118  0.469668  
P20     3        0.0                        42  0.380952  0.491507  
P21     3        NaN                         1  0.000000       NaN  
P22     3        0.0                        31  0.258065  0.444803

In [135]:
res = dfPatientsVect[dfPatientsVect['EPISODE'] == 3].groupby(by=['PATIENT','EPISODE'])
res.agg(['count','mean','std'])[['EMOTION_DISTRESSED', 'EMOTION_FUSTRATED', 'EMOTION_GUILTY', 'EMOTION_HAPPY','EMOTION_RELAXED', 'EMOTION_SAD']]\
.head(10)

EMOTION_DISTRESSED                     EMOTION_FUSTRATED  \
                             count      mean       std             count   
PATIENT EPISODE                                                            
P10     3                       35  0.228571  0.426043                35   
P11     3                       25  0.120000  0.331662                25   
P13     3                       52  0.211538  0.412384                52   
P15     3                       32  0.187500  0.396558                32   
P17     3                       40  0.175000  0.384808                40   
P19     3                        7  0.000000  0.000000                 7   
P2      3                       17  0.294118  0.469668                17   
P20     3                       42  0.261905  0.445001                42   
P21     3                        1  1.000000       NaN                 1   
P22     3                       31  0.387097  0.495138                31   

                                    EMOTION_GUILTY                      \
                     mean       std          count      mean       std   
PATIENT EPISODE                                                          
P10     3        0.314286  0.471008             35  0.228571  0.426043   
P11     3        0.280000  0.458258             25  0.360000  0.489898   
P13     3        0.326923  0.473665             52  0.211538  0.412384   
P15     3        0.281250  0.456803             32  0.312500  0.470929   
P17     3        0.250000  0.438529             40  0.275000  0.452203   
P19     3        0.428571  0.534522              7  0.285714  0.487950   
P2      3        0.058824  0.242536             17  0.235294  0.437237   
P20     3        0.214286  0.415300             42  0.190476  0.397437   
P21     3        0.000000       NaN              1  0.000000       NaN   
P22     3        0.096774  0.300537             31  0.096774  0.300537   

                EMOTION_HAPPY           EMOTION_RELAXED           EMOTION_SAD  \
                        count mean  std           count mean  std       count   
PATIENT EPISODE                                                                 
P10     3                  35    0  0.0              35    0  0.0          35   
P11     3                  25    0  0.0              25    0  0.0          25   
P13     3                  52    0  0.0              52    0  0.0          52   
P15     3                  32    0  0.0              32    0  0.0          32   
P17     3                  40    0  0.0              40    0  0.0          40   
P19     3                   7    0  0.0               7    0  0.0           7   
P2      3                  17    0  0.0              17    0  0.0          17   
P20     3                  42    0  0.0              42    0  0.0          42   
P21     3                   1    0  NaN               1    0  NaN           1   
P22     3                  31    0  0.0              31    0  0.0          31   

                                     
                     mean       std  
PATIENT EPISODE                      
P10     3        0.228571  0.426043  
P11     3        0.240000  0.435890  
P13     3        0.250000  0.437237  
P15     3        0.218750  0.420013  
P17     3        0.300000  0.464095  
P19     3        0.285714  0.487950  
P2      3        0.411765  0.507300  
P20     3        0.333333  0.477119  
P21     3        0.000000       NaN  
P22     3        0.419355  0.501610

In [139]:
r = res.agg(['count','mean','std'])[['EMOTION_DISTRESSED', 'EMOTION_FUSTRATED', 'EMOTION_GUILTY', 'EMOTION_HAPPY','EMOTION_RELAXED', 'EMOTION_SAD']].values

[502.0,
 4.654642462629491,
 nan,
 502.0,
 4.169016932098086,
 nan,
 502.0,
 4.174070521311128,
 nan,
 502.0,
 0.0,
 nan,
 502.0,
 0.0,
 nan,
 502.0,
 4.002270083961294,
 nan]